<a href="https://colab.research.google.com/github/alancarlosilva/BancoNoSQL/blob/master/Exercicio_Final_NASA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#importando as bibliotecas
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, column, substring
import pyspark.sql.functions as F
import pandas as pd
import csv


#localização dos arquivos no Databricks
data_a = "/FileStore/tables/NASA_Jul95.csv"
data_b = "/FileStore/tables/NASA_Aug95.csv"

In [0]:
#Iniciando a sessão Spark usando SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark - Nasa") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [0]:
#Definindo o tipo de arquivo como "csv", a primeira linha não vai como cabeçalho.
#Passado como parâmetro e exibindo os primeiros registros. As variáveis df_data_a e df_data_b conterá o DataFrame correspondente ao arquivo aberto
df_data_a = spark.read.format("csv")\
            .option("delimiter", " ")\
            .option("multilines", "true")\
            .option("quotes", '""')\
            .option("header", "false")\
            .load(data_a)\
            .toDF("host", "a", "b", "dias", "time_zone", "request", "http", "bytes")

df_data_b = spark.read.format("csv")\
            .option("delimiter", " ")\
            .option("multilines", "true")\
            .option("quotes", '""')\
            .option("header", "false")\
            .load(data_b)\
            .toDF("host", "a", "b", "dias", "time_zone", "request", "http", "bytes")

In [0]:
#mostrando os dez primeiros resultados de NASA_Jul95
df_data_a.show(10,truncate = False)
df_data_b.show(10,truncate = False)

+--------------------+---+---+---------------------+---------+------------------------------------------------------------+----+-----+
host |a |b |dias |time_zone|request |http|bytes|
+--------------------+---+---+---------------------+---------+------------------------------------------------------------+----+-----+
199.72.81.55 |- |- |[01/Jul/1995:00:00:01|-0400] |GET /history/apollo/ HTTP/1.0 |200 |6245 |
unicomp6.unicomp.net|- |- |[01/Jul/1995:00:00:06|-0400] |GET /shuttle/countdown/ HTTP/1.0 |200 |3985 |
199.120.110.21 |- |- |[01/Jul/1995:00:00:09|-0400] |GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0 |200 |4085 |
burger.letters.com |- |- |[01/Jul/1995:00:00:11|-0400] |GET /shuttle/countdown/liftoff.html HTTP/1.0 |304 |0 |
199.120.110.21 |- |- |[01/Jul/1995:00:00:11|-0400] |GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0|200 |4179 |
burger.letters.com |- |- |[01/Jul/1995:00:00:12|-0400] |GET /images/NASA-logosmall.gif HTTP/1.0 |304 |0 |
burger.letters.com |- |- |[01/Jul/1995:00:00:12|-0400] |GET /shuttle/countdown/video/livevideo.gif HTTP/1.0 |200 |0 |
205.212.115.106 |- |- |[01/Jul/1995:00:00:12|-0400] |GET /shuttle/countdown/countdown.html HTTP/1.0 |200 |3985 |
d104.aa.net |- |- |[01/Jul/1995:00:00:13|-0400] |GET /shuttle/countdown/ HTTP/1.0 |200 |3985 |
129.94.144.152 |- |- |[01/Jul/1995:00:00:13|-0400] |GET / HTTP/1.0 |200 |7074 |
+--------------------+---+---+---------------------+---------+------------------------------------------------------------+----+-----+
only showing top 10 rows

+---------------------------+---+---+---------------------+---------+------------------------------------------------------------+----+-----+
host |a |b |dias |time_zone|request |http|bytes|
+---------------------------+---+---+---------------------+---------+------------------------------------------------------------+----+-----+
in24.inetnebr.com |- |- |[01/Aug/1995:00:00:01|-0400] |GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0|200 |1839 |
uplherc.upl.com |- |- |[01/Aug/1995:00:00:07|-0400] |GET / HTTP/1.0 |304 |0 |
uplherc.upl.com |- |- |[01/Aug/1995:00:00:08|-0400] |GET /images/ksclogo-medium.gif HTTP/1.0 |304 |0 |
uplherc.upl.com |- |- |[01/Aug/1995:00:00:08|-0400] |GET /images/MOSAIC-logosmall.gif HTTP/1.0 |304 |0 |
uplherc.upl.com |- |- |[01/Aug/1995:00:00:08|-0400] |GET /images/USA-logosmall.gif HTTP/1.0 |304 |0 |
ix-esc-ca2-07.ix.netcom.com|- |- |[01/Aug/1995:00:00:09|-0400] |GET /images/launch-logo.gif HTTP/1.0 |200 |1713 |
uplherc.upl.com |- |- |[01/Aug/1995:00:00:10|-0400] |GET /images/WORLD-logosmall.gif HTTP/1.0 |304 |0 |
slppp6.intermind.net |- |- |[01/Aug/1995:00:00:10|-0400] |GET /history/skylab/skylab.html HTTP/1.0 |200 |1687 |
piweba4y.prodigy.com |- |- |[01/Aug/1995:00:00:10|-0400] |GET /images/launchmedium.gif HTTP/1.0 |200 |11853|
slppp6.intermind.net |- |- |[01/Aug/1995:00:00:11|-0400] |GET /history/skylab/skylab-small.gif HTTP/1.0 |200 |9202 |
+---------------------------+---+---+---------------------+---------+------------------------------------------------------------+----+-----+
only showing top 10 rows

In [0]:
#Vamos tirar algumas colunas desnecessárias em nosso data Frame utilizando spark.sql
df_data_a.createOrReplaceTempView("NASA_Jul95")
df_data_b.createOrReplaceTempView("NASA_Aug95")

df1 = spark.sql("Select host, dias, request, http, bytes from NASA_Jul95")
df2 = spark.sql("Select host, dias, request, http, bytes from NASA_Aug95")

#Vamos fazer um union dos dois datasets para respondermos as perguntas abaixo 
access_log = df1.union(df2)

In [0]:
#1. Número de hosts únicos.
#Vamos utlizar a função select com filtro em host e distinct
access_log.select("host").distinct().count()

Out[6]: 137979

In [0]:
#2. O total de erros 404.
#Existem algumas maneiras de fazermos esse filtro pelo código de erro 404, primeiro vamos usar código SQL
access_log.createOrReplaceTempView("Erro404")

spark.sql("Select * from Erro404 where http='404'").count()

#Segundo vamos usar filter
access_log.filter(access_log['http'].contains('404')).count()

Out[7]: 20871

In [0]:
#3. Os 5 URLs que mais causaram erro 404.
#Primeiro exemplo vamos usar SQL para mostrar as 5 URLs com mais erros
spark.sql("Select host, count(host) as qtde from Erro404 where http='404' group by host order by 2 desc limit 5").show(10, truncate=False)

#Segundo vamos utilizar funções
access_log.select('host').where("http='404'").groupBy('host').count().orderBy('count',ascending=False).limit(5).show(truncate=False)

+---------------------------+----+
host |qtde|
+---------------------------+----+
hoohoo.ncsa.uiuc.edu |251 |
piweba3y.prodigy.com |156 |
jbiagioni.npt.nuwc.navy.mil|132 |
piweba1y.prodigy.com |114 |
www-d4.proxy.aol.com |91 |
+---------------------------+----+

+---------------------------+-----+
host |count|
+---------------------------+-----+
hoohoo.ncsa.uiuc.edu |251 |
piweba3y.prodigy.com |156 |
jbiagioni.npt.nuwc.navy.mil|132 |
piweba1y.prodigy.com |114 |
www-d4.proxy.aol.com |91 |
+---------------------------+-----+

In [0]:
#4. Quantidade de erros 404 por dia.

spark.sql("Select substring(dias, 2,11) as dias, count('dias') from Erro404 where http='404' group by 1").show(10, truncate=False)


+-----------+-----------+
dias |count(dias)|
+-----------+-----------+
02/Jul/1995|291 |
21/Aug/1995|305 |
06/Aug/1995|373 |
16/Jul/1995|257 |
07/Aug/1995|537 |
11/Aug/1995|263 |
27/Jul/1995|336 |
07/Jul/1995|569 |
17/Jul/1995|406 |
15/Jul/1995|254 |
+-----------+-----------+
only showing top 10 rows

In [0]:
#5. O total de bytes retornados.
#Utilizando função select
access_log.select(F.sum('bytes')).show()

#Utilizando spark.sql
access_log.createOrReplaceTempView("TotalBytes")

spark.sql("Select sum(bytes) as total_bytes from TotalBytes").show(truncate=False)

+---------------+
 sum(bytes)|
+---------------+
6.5524319796E10|
+---------------+

+---------------+
total_bytes |
+---------------+
6.5524319796E10|
+---------------+